In [1]:
import json
import requests
import pandas as pd
import time
from datetime import datetime
import psycopg2
import numpy as np
import math
import email, smtplib, ssl
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

import import_ipynb
import tcg_bearer_token
import data_structures as ds

from psycopg2.extras import execute_values

importing Jupyter notebook from tcg_bearer_token.ipynb
importing Jupyter notebook from data_structures.ipynb


In [2]:
def send_prices(filename):
    subject = "Cardsphere Pricing CSV"
    body = "Beep boop here's your Cardsphere prices!"
    sender_email = "householdgamingmtg@gmail.com"
    receiver_email = "householdgamingmtg@gmail.com"
    password = ''

    # Create a multipart message and set headers
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject
#     message["Bcc"] = receiver_email  # Recommended for mass emails

    # Add body to email
    message.attach(MIMEText(body, "plain"))

    # Open PDF file in binary mode
    with open(filename, "rb") as attachment:
        # Add file as application/octet-stream
        # Email client can usually download this automatically as attachment
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())

    # Encode file in ASCII characters to send by email    
    encoders.encode_base64(part)

    # Add header as key/value pair to attachment part
    part.add_header(
        "Content-Disposition",
        f"attachment; filename= {filename}",
    )

    # Add attachment to message and convert message to string
    message.attach(part)
    text = message.as_string()

    # Log in to server using secure context and send email
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, text)

In [3]:
def bl_conversion_eqn(price):
    return price * (.85 / (1 + 3.1 * math.exp(-.96 * price)))

In [4]:
def create_buy_price(row):
    estim_returns= []
    for source in ['mm', 'ck', 'csi', 'ninety']:
        if row[source + 'buyprice'] != 0:
            if row[source + '_week_buy_avg'] > row[source + 'buyprice']:
                estim_returns.append(row[source + 'buyprice'])
            else:
                estim_returns.append(row[source + '_week_buy_avg'])
    estim_returns.sort(reverse=True)
    
    if len(estim_returns) >=2:
        top_price = estim_returns[0]
        second_price = estim_returns[1]
        bl_return = 0

        if top_price > second_price * 1.1:
            bl_return = second_price*1.1
        elif top_price <= second_price*1.1:
            bl_return = top_price
    elif len(estim_returns) == 1:
        bl_return = estim_returns[0] *.7
    elif len(estim_returns) == 0:
        bl_return = 0
    
    if bl_return == 0:
        return 0
    
    tcg_return = 0
    if row.lowprice + .99 > 5:
        if row.week_low_avg > row.lowprice:
            if row.lowprice + .99 < 50:
                tcg_return = (((row.lowprice+.99) * .8725) - .3) - .42
            elif row.lowprice + .99 >= 50:
                tcg_return = (((row.lowprice+.99) * .8725) - .3) - 3
        elif row.week_low_avg <= row.lowprice:
            if row.week_low_avg + .99 < 50:
                tcg_return = (((row.week_low_avg+.99) * .8725) - .3) - .42
            elif row.week_low_avg + .99 >= 50:
                tcg_return = (((row.week_low_avg+.99) * .8725) - .3) - 3
    
    my_bl_price = bl_conversion_eqn(bl_return)
            
    my_price = max(my_bl_price, tcg_return / 1.3)
    
    if my_price > 0 and my_price < .01:
        my_price = .01
    if my_price > 100:
        my_price = 100
    if my_price > tcg_return * 1.3 and tcg_return > 0:
        my_price = tcg_return * 1.3
    
    return my_price

In [5]:
def main():
    conn = psycopg2.connect(host="localhost", port=5432, database="", user="", password="")
    cur = conn.cursor()

    SQL = """SELECT COUNT(mmbuyprice) as mm_count FROM mmbuydata WHERE mmdate > NOW() - INTERVAL '2 week' GROUP BY uuid;"""
    mm_week_data_count = max(pd.read_sql_query(SQL, conn)['mm_count'].unique())

    SQL = """SELECT COUNT(csibuyprice) as csi_count FROM csibuydata WHERE csidate > NOW() - INTERVAL '2 week' GROUP BY uuid;"""
    csi_week_data_count = max(pd.read_sql_query(SQL, conn)['csi_count'].unique())

    SQL = """SELECT COUNT(ckbuyprice) as ck_count FROM ckbuydata WHERE ckdate > NOW() - INTERVAL '2 week' GROUP BY uuid;"""
    ck_week_data_count = max(pd.read_sql_query(SQL, conn)['ck_count'].unique())

    SQL = """SELECT COUNT(ninetybuyprice) as ninety_count FROM ninetybuydata WHERE ninetydate > NOW() - INTERVAL '2 week' GROUP BY uuid;"""
    ninety_week_data_count = max(pd.read_sql_query(SQL, conn)['ninety_count'].unique())


    SQL = "SELECT uuid, tcgplayerproductid FROM cardinfo WHERE setname NOT IN ('Legends', 'Urza''s Legacy', 'Urza''s Destiny', 'Urza''s Saga');"
    df = pd.read_sql_query(SQL, conn)


    SQL = """SELECT tcgplayerproductid, AVG(lowprice::numeric) as week_low_avg, AVG(marketprice::numeric) as week_market_avg FROM tcgpricedata WHERE tcgdate > NOW() - INTERVAL '2 week' GROUP BY tcgplayerproductid;"""
    tcg_week_avg_df = pd.read_sql_query(SQL, conn)
    df = df.merge(tcg_week_avg_df, how='left', on='tcgplayerproductid')

    SQL = 'SELECT tcgplayerproductid, lowprice::numeric, marketprice::numeric FROM tcgpricedata WHERE tcgdate = (SELECT MAX(tcgdate) FROM tcgpricedata) AND foil = FALSE;'
    tcg_current_df = pd.read_sql_query(SQL, conn)
    df = df.merge(tcg_current_df, how='left', on='tcgplayerproductid')


    SQL = """SELECT uuid, AVG(mmbuyprice::numeric) as mm_week_buy_avg, COUNT(mmbuyprice) as mm_count FROM mmbuydata WHERE mmdate > NOW() - INTERVAL '2 week' GROUP BY uuid;"""
    mm_week_avg_df = pd.read_sql_query(SQL, conn)
    mm_week_avg_df['mm_week_buy_avg'] = mm_week_avg_df['mm_week_buy_avg'] * (mm_week_avg_df['mm_count'] / mm_week_data_count)
    df = df.merge(mm_week_avg_df, how='left', on='uuid')

    SQL = 'SELECT uuid, mmbuyprice::numeric FROM mmbuydata WHERE mmdate = (SELECT MAX(mmdate) FROM mmbuydata);'
    mm_current_df = pd.read_sql_query(SQL, conn)
    df = df.merge(mm_current_df, how='left', on='uuid')


    SQL = """SELECT uuid, AVG(csibuyprice::numeric) as csi_week_buy_avg, COUNT(csibuyprice) as csi_count FROM csibuydata WHERE csidate > NOW() - INTERVAL '2 week' GROUP BY uuid;"""
    csi_week_avg_df = pd.read_sql_query(SQL, conn)
    csi_week_avg_df['csi_week_buy_avg'] = csi_week_avg_df['csi_week_buy_avg'] * (csi_week_avg_df['csi_count'] / csi_week_data_count)
    df = df.merge(csi_week_avg_df, how='left', on='uuid')

    SQL = 'SELECT uuid, csibuyprice::numeric FROM csibuydata WHERE csidate = (SELECT MAX(csidate) FROM csibuydata);'
    csi_current_df = pd.read_sql_query(SQL, conn)
    df = df.merge(csi_current_df, how='left', on='uuid')


    SQL = """SELECT uuid, AVG(ckbuyprice::numeric) as ck_week_buy_avg, COUNT(ckbuyprice) as ck_count FROM ckbuydata WHERE ckdate > NOW() - INTERVAL '2 week' GROUP BY uuid;"""
    ck_week_avg_df = pd.read_sql_query(SQL, conn)
    ck_week_avg_df['ck_week_buy_avg'] = ck_week_avg_df['ck_week_buy_avg'] * (ck_week_avg_df['ck_count'] / ck_week_data_count)
    df = df.merge(ck_week_avg_df, how='left', on='uuid')

    SQL = 'SELECT uuid, ckbuyprice::numeric FROM ckbuydata WHERE ckdate = (SELECT MAX(ckdate) FROM ckbuydata);'
    ck_current_df = pd.read_sql_query(SQL, conn)
    df = df.merge(ck_current_df, how='left', on='uuid')


    SQL = """SELECT uuid, AVG(ninetybuyprice::numeric) as ninety_week_buy_avg, COUNT(ninetybuyprice) as ninety_count FROM ninetybuydata WHERE ninetydate > NOW() - INTERVAL '2 week' GROUP BY uuid;"""
    ninety_week_avg_df = pd.read_sql_query(SQL, conn)
    ninety_week_avg_df['ninety_week_buy_avg'] = ninety_week_avg_df['ninety_week_buy_avg'] * (ninety_week_avg_df['ninety_count'] / ninety_week_data_count)
    df = df.merge(ninety_week_avg_df, how='left', on='uuid')

    SQL = 'SELECT uuid, ninetybuyprice::numeric FROM ninetybuydata WHERE ninetydate = (SELECT MAX(ninetydate) FROM ninetybuydata);'
    ninety_current_df = pd.read_sql_query(SQL, conn)
    df = df.merge(ninety_current_df, how='left', on='uuid')


    conn.close()
    cur.close()


    df.fillna(0, inplace=True)

    df['buy_price'] = df.apply(create_buy_price, axis=1)
    df = df.loc[df['buy_price'] > 0]

    conn = psycopg2.connect(host="localhost", port=5432, database="", user="", password="")
    cur = conn.cursor()

    wants = pd.DataFrame(columns=['Quantity', 'Name', 'Sets', 'Conditions', 'Languages', 'Finishes', 'Paused', 'Tags', 'Offer', 'Limit'])
    wants['Tags'] = df['buy_price']

    for index, row in df.iterrows():
        card_info = ds.get_properties(cur, row.uuid, 'cs')
        wants.at[index, 'Name'] = card_info[0]
        wants.at[index, 'Sets'] = card_info[1]
        wants.at[index, 'Limit'] = row.buy_price

    wants['Tags'] = np.nan
    wants['Quantity'] = 8
    wants['Conditions'] = 'NM'
    wants['Languages'] = 'EN'
    wants['Finishes'] = 'N'
    wants['Offer'] = 1000

    conn.close()
    cur.close()

    wants.to_csv(r'cs_pricing.csv', index=False)
    #send_prices('cs_pricing.csv')

In [6]:
if __name__ == '__main__':
    main()